# Testing Notebook

## Start container mssqlsrv1

Use docker

In [2]:
# -- Start mssqlsrv1
$null = docker start mssqlsrv1
docker ps

CONTAINER ID   IMAGE               COMMAND                  CREATED       STATUS                  PORTS                                       NAMES
e7ff4a53d0dd   mssql:2019-latest   "/opt/mssql/bin/perm…"   2 weeks ago   Up Less than a second   0.0.0.0:8433->1433/tcp, :::8433->1433/tcp   mssqlsrv1


## Create a TestingDB on MSSQL Server

Use dbatools, what else?

In [32]:
# -- Create new database 'TestingDB'
$database = 'TestingDB'
if(-not(Get-InstalledModule -Name dbatools)){
    Install-Module -Name dbatools
}
if(-not(Get-Module -Name dbatools)){
    Import-Module -Name dbatools
}
if([String]::IsNullOrEmpty($sqlcred)){
    $secStringPassword = ConvertTo-SecureString 'yourStrong(!)Password' -AsPlainText -Force
    $sqlcred = New-Object System.Management.Automation.PSCredential ('sa', $secStringPassword)
}
$mssqlsrv = Connect-DbaInstance -SqlInstance 'localhost:8433' -SqlCredential $sqlcred
$dbexists = Get-DbaDatabase -SqlInstance $mssqlsrv  -Database $database
if($dbexists){
    $dbexists
}else{
    New-DbaDatabase -SqlInstance $mssqlsrv -Name $database
}


ComputerName       : localhost
InstanceName       : MSSQLSERVER
SqlInstance        : mssqlsrv1
Name               : TestingDB
Status             : Normal
IsAccessible       : True
RecoveryModel      : Full
LogReuseWaitStatus : Nothing
SizeMB             : 16
Compatibility      : Version150
Collation          : SQL_Latin1_General_CP1_CI_AS
Owner              : sa
LastFullBackup     : 01/01/0001 00:00:00
LastDiffBackup     : 01/01/0001 00:00:00
LastLogBackup      : 01/01/0001 00:00:00




## Create new Testing Table

Use dbatools, what else?

In [34]:
# -- Create new table 'Test_Case1'
$table    = 'Test_Case1'
$database = 'TestingDB'
$Columns  =  @()
$Columns  += @{
    Name      = 'ID'
    Type      = 'int'
    Identity  = $true
}
$Columns  += @{
    Name      = 'TimeStamp'
    Type      = 'datetime'
    Nullable  =  $false
    Default   =  'GETDATE()'
}
$Columns  += @{
    Name      = 'Testcase'
    Type      = 'Nvarchar'
    MaxLength = 4000
    Nullable  =  $true
}
$Columns  += @{
    Name      = 'How to Test'
    Type      = 'Nvarchar'
    MaxLength = 4000
    Nullable  =  $true
}
$Columns  += @{
    Name      = 'Expectet Result'
    Type      = 'Nvarchar'
    MaxLength = 4000
    Nullable  =  $true
}
$Columns  += @{
    Name      = 'Received Result'
    Type      = 'Nvarchar'
    MaxLength = 4000
    Nullable  =  $true
}
$Columns  += @{
    Name      = 'Pass/Fail'
    Type      = 'Nvarchar'
    MaxLength = 10
    Nullable  =  $true
}
$tblexists = Get-DbaDbTable -SqlInstance $mssqlsrv -Database $database -Table $table
if($tblexists){
    $tblexists
}else{
    New-DbaDbTable -SqlInstance $mssqlsrv -Database $database -Name $table -ColumnMap $Columns
}


ComputerName          : localhost
InstanceName          : MSSQLSERVER
SqlInstance           : mssqlsrv1
Database              : TestingDB
Schema                : dbo
Name                  : Test_Case1
IndexSpaceUsed        : 0
DataSpaceUsed         : 0
RowCount              : 0
HasClusteredIndex     : False
IsFileTable           : False
IsMemoryOptimized     : False
IsPartitioned         : False
FullTextIndex         : 
ChangeTrackingEnabled : False




## PreRequisites for Testing

Create and start container pyhost1 and pyhost2

In [38]:
# -- Start container
$null = docker run -e TZ="Europe/Zurich" -it -v fileshare:/shared-volume --network custom --hostname pyhost1 --name pyhost1 -d pyhost:1.0.0
$null = docker run -e TZ="Europe/Zurich" -it -v fileshare:/shared-volume --network custom --hostname pyhost2 --name pyhost2 -d pyhost:1.0.0
docker ps

CONTAINER ID   IMAGE               COMMAND                  CREATED         STATUS                  PORTS                                       NAMES
4e1b93206b2e   pyhost:1.0.0        "/bin/bash"              2 seconds ago   Up Less than a second                                               pyhost2
a97f3a3f1893   pyhost:1.0.0        "/bin/bash"              5 seconds ago   Up 2 seconds                                                        pyhost1
e7ff4a53d0dd   mssql:2019-latest   "/opt/mssql/bin/perm…"   2 weeks ago     Up About an hour        0.0.0.0:8433->1433/tcp, :::8433->1433/tcp   mssqlsrv1


## Use Case 1

In [58]:
$container = @('pyhost1', 'pyhost2')
foreach($item in $container){
    [PSCustomObject]@{
        Hostname  = docker inspect --format='{{.Config.Hostname}}' $item 
        IPAddress = docker inspect --format='{{.NetworkSettings.Networks.custom.IPAddress}}' $item 
        Path      = docker inspect --format='{{.Path}}' $item 
    }
}

Hostname IPAddress  Path
-------- ---------  ----
pyhost1  172.18.0.3 /bin/bash
pyhost2  172.18.0.4 /bin/bash



In [60]:
<#
    'ID',
    'TimeStamp',
    'Testcase',
    'How to Test',
    'Expectet Result',
    'Received Result',
    'Pass/Fail'
#>
$SqlQuery   = @"
SELECT * FROM [$database].[dbo].[$table]
GO
"@  
Invoke-DbaQuery -SqlInstance $mssqlsrv -Database $DBname -Query $SqlQuery